In [1]:
!pip install -r ./requirements.txt

  Cloning https://github.com/IndoNLP/nusa-crowd.git (to revision 7748513d20331e72f9969f94f5d43c7f2d4a59a5) to /tmp/pip-install-ktb96etd/nusacrowd_ae8d3bdaddb64e9484e17d1110d681ee
  Running command git clone --filter=blob:none -q https://github.com/IndoNLP/nusa-crowd.git /tmp/pip-install-ktb96etd/nusacrowd_ae8d3bdaddb64e9484e17d1110d681ee
  Running command git rev-parse -q --verify 'sha^7748513d20331e72f9969f94f5d43c7f2d4a59a5'
  Running command git fetch -q https://github.com/IndoNLP/nusa-crowd.git 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Running command git checkout -q 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Resolved https://github.com/IndoNLP/nusa-crowd.git to commit 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import transformers
import evaluate
import torch
import operator
import ast
import json
import re
import sys
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch.nn as nn

from multiprocessing import cpu_count
from evaluate import load
from nusacrowd import NusantaraConfigHelper
from torch.utils.data import DataLoader
from datetime import datetime
from huggingface_hub import notebook_login

from datasets import (
    load_dataset, 
    load_from_disk,
    Dataset,
    DatasetDict
)
from transformers import (
    BigBirdTokenizerFast,
    BigBirdForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    BertForSequenceClassification,
    BertForQuestionAnswering,
    AutoModel, 
    BertTokenizerFast,
    AutoTokenizer, 
    AutoModel, 
    BertTokenizer, 
    BertForPreTraining,
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering
)

In [3]:
MODEL_NAME = "indolem/indobert-base-uncased"
SEED = 42
EPOCH = 1
BATCH_SIZE = 16
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 1e-5
MAX_LENGTH = 400
STRIDE = 100
LOGGING_STEPS = 50
WARMUP_RATIO = 0.06
WEIGHT_DECAY = 0.01
# Untuk mempercepat training, saya ubah SAMPLE menjadi 100.
# Bila mau menggunakan keseluruhan data, gunakan: 
# SAMPLE = sys.maxsize
SAMPLE = 10

In [4]:
conhelps = NusantaraConfigHelper()
data_idk = conhelps.filtered(lambda x: 'idk_mrc' in x.dataset_name)[0].load_dataset()

df_train = pd.DataFrame(data_idk['train'])
df_validation = pd.DataFrame(data_idk['validation'])

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)

for i in range(len(df_validation['context'])):
    for j in df_validation["qas"][i]:
        if len(j['answers']) != 0:
            new_df_val = new_df_val.append({'context': df_validation["context"][i], 'question': j['question'], 'answer': {"text": j['answers'][0]['text'], "answer_start": j['answers'][0]['answer_start'], "answer_end": j['answers'][0]['answer_start'] + len(j['answers'][0]['text'])}}, ignore_index=True)
        else:
            new_df_val = new_df_val.append({'context': df_validation["context"][i], 'question': j['question'], 'answer': {"text": str(), "answer_start": 0, "answer_end": 0}}, ignore_index=True)

cols = ['context', 'question', 'answer']
new_df_train = pd.DataFrame(columns=cols)

for i in range(len(df_train['context'])):
    for j in df_train["qas"][i]:
        if len(j['answers']) != 0:
            new_df_train = new_df_train.append({'context': df_train["context"][i], 'question': j['question'], 'answer': {"text": j['answers'][0]['text'], "answer_start": j['answers'][0]['answer_start'], "answer_end": j['answers'][0]['answer_start'] + len(j['answers'][0]['text'])}}, ignore_index=True)
        else:
            new_df_train = new_df_train.append({'context': df_train["context"][i], 'question': j['question'], 'answer': {"text": str(), "answer_start": 0, "answer_end": 0}}, ignore_index=True)

train_dataset = Dataset.from_dict(new_df_train)
validation_dataset = Dataset.from_dict(new_df_val)

data_idk = DatasetDict({"train": train_dataset, "validation": validation_dataset})
data_idk

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 9332
    })
    validation: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 764
    })
})

In [5]:
data_tydiqa = load_dataset("tydiqa", 'secondary_task')

df_train = pd.DataFrame(data_tydiqa['train'])
df_validation = pd.DataFrame(data_tydiqa['validation'])

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)

for i in range(len(df_validation['context'])):
    new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                    'question': df_validation["question"][i], 
                                    'answer': {"text": df_validation["answers"][i]['text'][0], 
                                    "answer_start": df_validation["answers"][i]['answer_start'][0], 
                                    "answer_end": df_validation["answers"][i]['answer_start'][0] + len(df_validation["answers"][i]['text'][0])}}, 
                                   ignore_index=True)

cols = ['context', 'question', 'answer']
new_df_train = pd.DataFrame(columns=cols)

for i in range(len(df_train['context'])):
    new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                    'question': df_train["question"][i], 
                                    'answer': {"text": df_train["answers"][i]['text'][0], 
                                    "answer_start": df_train["answers"][i]['answer_start'][0], 
                                    "answer_end": df_train["answers"][i]['answer_start'][0] + len(df_train["answers"][i]['text'][0])}}, 
                                   ignore_index=True)
    
train_dataset = Dataset.from_dict(new_df_train)
validation_dataset = Dataset.from_dict(new_df_val)

data_tydiqa = DatasetDict({"train": train_dataset, "validation": validation_dataset})
data_tydiqa

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 49881
    })
    validation: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 5077
    })
})

In [6]:
conhelps = NusantaraConfigHelper()
data_squad_id = conhelps.filtered(lambda x: 'squad_id' in x.dataset_name)[0].load_dataset()

df_train = pd.DataFrame(data_squad_id['train'])
df_validation = pd.DataFrame(data_squad_id['validation'])

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_validation['context']))):
    new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                    'question': df_validation["question"][i], 
                                    'answer': {"text": eval(df_validation["answer"][i][0])['text'], 
                                    "answer_start": eval(df_validation["answer"][i][0])['answer_start'], 
                                    "answer_end": eval(df_validation["answer"][i][0])['answer_end']}}, 
                                   ignore_index=True)
    
cols = ['context', 'question', 'answer']
new_df_train = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_train['context']))):
    new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                    'question': df_train["question"][i], 
                                    'answer': {"text": eval(df_train["answer"][i][0])['text'], 
                                    "answer_start": eval(df_train["answer"][i][0])['answer_start'], 
                                    "answer_end": eval(df_train["answer"][i][0])['answer_end']}}, 
                                   ignore_index=True)

train_dataset = Dataset.from_dict(new_df_train)
validation_dataset = Dataset.from_dict(new_df_val)

data_squad_id = DatasetDict({"train": train_dataset, "validation": validation_dataset})
data_squad_id

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████| 130319/130319 [08:38<00:00, 251.15it/s]


DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 11873
    })
})

In [7]:
squad_id_train_json = pd.DataFrame(data_squad_id['train']).to_json(orient='records', lines=True)
squad_id_val_json = pd.DataFrame(data_squad_id['validation']).to_json(orient='records', lines=True)

idk_mrc_train_json = pd.DataFrame(data_idk['train']).to_json(orient='records', lines=True)
idk_mrc_val_json = pd.DataFrame(data_idk['validation']).to_json(orient='records', lines=True)

tydiqa_train_json = pd.DataFrame(data_tydiqa['train']).to_json(orient='records', lines=True)
tydiqa_val_json = pd.DataFrame(data_tydiqa['validation']).to_json(orient='records', lines=True)

with open('squad_id_train_reformatted.jsonl', 'w') as f:
    f.write(squad_id_train_json)
    
with open('squad_id_val_reformatted.jsonl', 'w') as f:
    f.write(squad_id_val_json)
    
with open('idk_mrc_train_reformatted.jsonl', 'w') as f:
    f.write(idk_mrc_train_json)

with open('idk_mrc_val_reformatted.jsonl', 'w') as f:
    f.write(idk_mrc_val_json)
    
with open('tydiqa_train_reformatted.jsonl', 'w') as f:
    f.write(tydiqa_train_json)
    
with open('tydiqa_val_reformatted.jsonl', 'w') as f:
    f.write(tydiqa_val_json)

In [8]:
#with open('squad_id_train_reformatted.jsonl', 'rb') as f:  
#      squad_id_train_json = json.load(f)

#with open('squad_id_val_reformatted.jsonl', 'rb') as f:
#    squad_id_val_json = json.load(f)
    
#with open('idk_mrc_train_reformatted.jsonl', 'rb') as f:
#    idk_mrc_train_json = json.load(f)

#with open('idk_mrc_val_reformatted.jsonl', 'rb') as f:
#    idk_mrc_val_json = json.load(f)
    
#with open('tydiqa_train_reformatted.jsonl', 'rb') as f:
#    tydiqa_train_json = json.load(f)
    
#with open('tydiqa_val_reformatted.jsonl', 'rb') as f:
#    tydiqa_val_json = json.load(f)

In [9]:
squad_id_train_df = pd.DataFrame(data_squad_id['train'])
squad_id_validation_df = pd.DataFrame(data_squad_id['validation'])

data_idk_train_df = pd.DataFrame(data_idk['train'])
data_idk_validation_df = pd.DataFrame(data_idk['validation'])

data_tydiqa_train_df = pd.DataFrame(data_tydiqa['train'])
data_tydiqa_validation_df = pd.DataFrame(data_tydiqa['validation'])

In [10]:
squad_id_all_df = pd.DataFrame(pd.concat([squad_id_train_df, squad_id_validation_df]))
data_idk_all_df = pd.DataFrame(pd.concat([data_idk_train_df, data_idk_validation_df]))
data_tydiqa_all_df = pd.DataFrame(pd.concat([data_tydiqa_train_df, data_tydiqa_validation_df]))

In [11]:
def number_of(data, nama, slices):
    num = 1
    for i in range(len(data[slices]) - 1):
      curr = data[slices][i]
      next = data[slices][i+1]
      if curr != next:
        num += 1
    print(f"{str(nama)} memiliki sejumlah {num} {slices}.")
    return num

In [12]:
x = number_of(data=squad_id_train_df, nama="SQUAD-ID Train", slices='context')
y = number_of(data=squad_id_validation_df, nama="SQUAD-ID Validation", slices='context')
print(f"SQUAD-ID All memiliki sejumlah {x+y} artikel.")
print()
x = number_of(data=data_idk_train_df, nama="IDK-MRC Train", slices='context')
y = number_of(data=data_idk_validation_df, nama="IDK-MRC Validation", slices='context')
print(f"IDK-MRC All memiliki sejumlah {x+y} artikel.")
print()
x = number_of(data=data_tydiqa_train_df, nama="TYDIQA Train", slices='context')
y = number_of(data=data_tydiqa_validation_df, nama="TYDIQA Validation", slices='context')
print(f"TYDIQA All memiliki sejumlah {x+y} artikel.")

SQUAD-ID Train memiliki sejumlah 19033 context.
SQUAD-ID Validation memiliki sejumlah 1204 context.
SQUAD-ID All memiliki sejumlah 20237 artikel.

IDK-MRC Train memiliki sejumlah 3528 context.
IDK-MRC Validation memiliki sejumlah 348 context.
IDK-MRC All memiliki sejumlah 3876 artikel.

TYDIQA Train memiliki sejumlah 49865 context.
TYDIQA Validation memiliki sejumlah 5074 context.
TYDIQA All memiliki sejumlah 54939 artikel.


In [13]:
x = number_of(data=squad_id_train_df, nama="SQUAD-ID Train", slices='question')
y = number_of(data=squad_id_validation_df, nama="SQUAD-ID Validation", slices='question')
print(f"SQUAD-ID All memiliki sejumlah {x+y} question.")
print()
x = number_of(data=data_idk_train_df, nama="IDK-MRC Train", slices='question')
y = number_of(data=data_idk_validation_df, nama="IDK-MRC Validation", slices='question')
print(f"IDK-MRC All memiliki sejumlah {x+y} question.")
print()
x = number_of(data=data_tydiqa_train_df, nama="TYDIQA Train", slices='question')
y = number_of(data=data_tydiqa_validation_df, nama="TYDIQA Validation", slices='question')
print(f"TYDIQA All memiliki sejumlah {x+y} question.")

SQUAD-ID Train memiliki sejumlah 130301 question.
SQUAD-ID Validation memiliki sejumlah 11871 question.
SQUAD-ID All memiliki sejumlah 142172 question.

IDK-MRC Train memiliki sejumlah 9327 question.
IDK-MRC Validation memiliki sejumlah 763 question.
IDK-MRC All memiliki sejumlah 10090 question.

TYDIQA Train memiliki sejumlah 49881 question.
TYDIQA Validation memiliki sejumlah 5077 question.
TYDIQA All memiliki sejumlah 54958 question.


In [14]:
print(f"Banyaknya passage SQUAD-ID Train: {squad_id_train_df.shape[0]}")
print(f"Banyaknya passage SQUAD-ID Validation: {squad_id_validation_df.shape[0]}")
print(f"Banyaknya passage SQUAD-ID All: {squad_id_all_df.shape[0]}")
print()
print(f"Banyaknya passage IDK-MRC Train: {data_idk_train_df.shape[0]}")
print(f"Banyaknya passage IDK-MRC Validation: {data_idk_validation_df.shape[0]}")
print(f"Banyaknya passage IDK-MR All: {data_idk_all_df.shape[0]}")
print()
print(f"Banyaknya passage TYDIQA Train: {data_tydiqa_train_df.shape[0]}")
print(f"Banyaknya passage TYDIQA Validation: {data_tydiqa_validation_df.shape[0]}")
print(f"Banyaknya passage TYDIQA All: {data_tydiqa_all_df.shape[0]}")

Banyaknya passage SQUAD-ID Train: 130319
Banyaknya passage SQUAD-ID Validation: 11873
Banyaknya passage SQUAD-ID All: 142192

Banyaknya passage IDK-MRC Train: 9332
Banyaknya passage IDK-MRC Validation: 764
Banyaknya passage IDK-MR All: 10096

Banyaknya passage TYDIQA Train: 49881
Banyaknya passage TYDIQA Validation: 5077
Banyaknya passage TYDIQA All: 54958


In [15]:
def number_of_answer(data, nama):
    unanswered = 0
    answered = 0
    for i in data['answer']:
        if len(i['text']) == 0:
            unanswered += 1
        else: answered += 1
    print(f"{str(nama)} memiliki answered: {answered} dan unanswered: {unanswered}.")
    #return (answered, unanswered)

In [16]:
number_of_answer(data=squad_id_train_df, nama="SQUAD-ID Train")
number_of_answer(data=squad_id_validation_df, nama="SQUAD-ID Validation")
number_of_answer(data=squad_id_all_df, nama="SQUAD-ID All")
print()
number_of_answer(data=data_idk_train_df, nama="IDK-MRC Train")
number_of_answer(data=data_idk_validation_df, nama="IDK-MRC Validation")
number_of_answer(data=data_idk_all_df, nama="IDK-MRC All")
print()
number_of_answer(data=data_tydiqa_train_df, nama="TYDIQA Train")
number_of_answer(data=data_tydiqa_validation_df, nama="TYDIQA Validation")
number_of_answer(data=data_tydiqa_all_df, nama="TYDIQA All")

SQUAD-ID Train memiliki answered: 130310 dan unanswered: 9.
SQUAD-ID Validation memiliki answered: 11858 dan unanswered: 15.
SQUAD-ID All memiliki answered: 142168 dan unanswered: 24.

IDK-MRC Train memiliki answered: 5042 dan unanswered: 4290.
IDK-MRC Validation memiliki answered: 382 dan unanswered: 382.
IDK-MRC All memiliki answered: 5424 dan unanswered: 4672.

TYDIQA Train memiliki answered: 49881 dan unanswered: 0.
TYDIQA Validation memiliki answered: 5077 dan unanswered: 0.
TYDIQA All memiliki answered: 54958 dan unanswered: 0.


In [17]:
def average_length(data, nama):
    data['avg_length_context'] = data["context"].apply(lambda x: len(x.split()))
    data['avg_length_question'] = data["question"].apply(lambda x: len(x.split()))
    data['avg_length_answer'] = data["answer"].apply(lambda x: len(x['text'].split()))

    print(f"{str(nama)} memiliki rerata panjang context: {data['avg_length_context'].mean()}")
    print(f"{str(nama)} memiliki rerata panjang question: {data['avg_length_question'].mean()}")
    print(f"{str(nama)} memiliki rerata panjang answer: {data['avg_length_answer'].mean()}")

In [18]:
average_length(data=squad_id_train_df, nama="SQUAD-ID Train")
average_length(data=squad_id_validation_df, nama="SQUAD-ID Validation")
average_length(data=squad_id_all_df, nama="SQUAD-ID All")
print()
average_length(data=data_idk_train_df, nama="IDK-MRC Train")
average_length(data=data_idk_validation_df, nama="IDK-MRC Validation")
average_length(data=data_idk_all_df, nama="IDK-MRC All")
print()
average_length(data=data_tydiqa_train_df, nama="TYDIQA Train")
average_length(data=data_tydiqa_validation_df, nama="TYDIQA Validation")
average_length(data=data_tydiqa_all_df, nama="TYDIQA All")

SQUAD-ID Train memiliki rerata panjang context: 107.38208549789363
SQUAD-ID Train memiliki rerata panjang question: 8.704970111802577
SQUAD-ID Train memiliki rerata panjang answer: 2.9721606212447917
SQUAD-ID Validation memiliki rerata panjang context: 113.82860271203572
SQUAD-ID Validation memiliki rerata panjang question: 8.797355344057946
SQUAD-ID Validation memiliki rerata panjang answer: 3.117661922007917
SQUAD-ID All memiliki rerata panjang context: 107.92036823449983
SQUAD-ID All memiliki rerata panjang question: 8.712684257904804
SQUAD-ID All memiliki rerata panjang answer: 2.9843099471137617

IDK-MRC Train memiliki rerata panjang context: 79.81986712387484
IDK-MRC Train memiliki rerata panjang question: 5.736283754822117
IDK-MRC Train memiliki rerata panjang answer: 2.5969781397342477
IDK-MRC Validation memiliki rerata panjang context: 77.66623036649214
IDK-MRC Validation memiliki rerata panjang question: 6.18848167539267
IDK-MRC Validation memiliki rerata panjang answer: 2.88

In [19]:
def vocab_size(data, nama):
    vocab = []
    for i in data["context"]:
        vocab.append(i.lower().split())
    for i in data["question"]:
        vocab.append(i.lower().split())
    for i in data["answer"]:
        vocab.append(i['text'].lower().split())
    print(f"{str(nama)} memiliki vocabulary size: {len(np.unique(vocab))}")

In [20]:
vocab_size(data=squad_id_train_df, nama="SQUAD-ID Train")
vocab_size(data=squad_id_validation_df, nama="SQUAD-ID Validation")
vocab_size(data=squad_id_all_df, nama="SQUAD-ID All")
print()
vocab_size(data=data_idk_train_df, nama="IDK-MRC Train")
vocab_size(data=data_idk_validation_df, nama="IDK-MRC Validation")
vocab_size(data=data_idk_all_df, nama="IDK-MRC All")
print()
vocab_size(data=data_tydiqa_train_df, nama="TYDIQA Train")
vocab_size(data=data_tydiqa_validation_df, nama="TYDIQA Validation")
vocab_size(data=data_tydiqa_all_df, nama="TYDIQA All")

/usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:272: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


SQUAD-ID Train memiliki vocabulary size: 226343
SQUAD-ID Validation memiliki vocabulary size: 20642
SQUAD-ID All memiliki vocabulary size: 245497

IDK-MRC Train memiliki vocabulary size: 16533
IDK-MRC Validation memiliki vocabulary size: 1474
IDK-MRC All memiliki vocabulary size: 17967

TYDIQA Train memiliki vocabulary size: 125074
TYDIQA Validation memiliki vocabulary size: 13962
TYDIQA All memiliki vocabulary size: 137934


In [21]:
def statistics_length(data, nama):
    under_hundred = []
    _101_to_150 = []
    _151_to_200 = []
    _201_to_250 = []
    _251_to_300 = []
    _over_301 = []
    
    q_one_to_five = []
    q_six_to_ten = []
    q_eleven_to_fifteen = []
    q_sixteen_to_twenty = []
    q_over_twenty = []
    
    a_zero = []
    a_one_to_five = []
    a_six_to_ten = []
    a_eleven_to_fifteen = []
    a_sixteen_to_twenty = []
    a_over_twenty = []

    for i in data['context']:
        len_current_passage = len(i.split())
        if len_current_passage <= 100: 
            under_hundred.append(len_current_passage)
        elif len_current_passage >= 101 & len_current_passage <= 150:
            _101_to_150.append(len_current_passage)
        elif len_current_passage >= 151 & len_current_passage <= 200:
            _151_to_200.append(len_current_passage)
        elif len_current_passage >= 201 & len_current_passage <= 250:
            _201_to_250.append(len_current_passage)
        elif len_current_passage >= 251 & len_current_passage <= 300:
            _251_to_300.append(len_current_passage)
        elif len_current_passage >= 301:
            _over_301.append(len_current_passage)
    
    for i in data['question']:
        len_current_question = len(i.split())
        if len_current_question <= 5: 
            q_one_to_five.append(len_current_question)
        elif len_current_question >= 6 & len_current_question <= 10:
            q_six_to_ten.append(len_current_question)
        elif len_current_question >= 11 & len_current_question <= 15:
            q_eleven_to_fifteen.append(len_current_question)
        elif len_current_question >= 16 & len_current_question <= 20:
            q_sixteen_to_twenty.append(len_current_question)
        elif len_current_question >= 21: 
            q_over_twenty.append(len_current_question)
    
    for i in data['answer']:
        len_current_answer = len(i['text'].split())
        if len_current_answer == 0:
            a_zero.append(len_current_answer)
        elif len_current_answer <= 5: 
            a_one_to_five.append(len_current_answer)
        elif len_current_answer >= 6 & len_current_answer <= 10:
            a_six_to_ten.append(len_current_answer)
        elif len_current_answer >= 11 & len_current_answer <= 15:
            a_eleven_to_fifteen.append(len_current_answer)
        elif len_current_answer >= 16 & len_current_answer <= 20:
            a_sixteen_to_twenty.append(len_current_answer)
        elif len_current_answer >= 21:
            a_over_twenty.append(len_current_answer)
    
    print(f"{str(nama)} memiliki banyak context: {len(under_hundred)} dengan panjang < 100")
    print(f"{str(nama)} memiliki banyak context: {len(_101_to_150)} dengan panjang 101 <= x <= 150")
    print(f"{str(nama)} memiliki banyak context: {len(_151_to_200)} dengan panjang 151 <= x <= 200")
    print(f"{str(nama)} memiliki banyak context: {len(_201_to_250)} dengan panjang 201 <= x <= 250")
    print(f"{str(nama)} memiliki banyak context: {len(_251_to_300)} dengan panjang 251 <= x <= 300")
    print(f"{str(nama)} memiliki banyak context: {len(_over_301)} dengan panjang > 300")
    print("=" * 100)
    print(f"{str(nama)} memiliki banyak question: {len(q_one_to_five)} dengan panjang 1 <= x <= 5")
    print(f"{str(nama)} memiliki banyak question: {len(q_six_to_ten)} dengan panjang 6 <= x <= 10")
    print(f"{str(nama)} memiliki banyak question: {len(q_eleven_to_fifteen)} dengan panjang 11 <= x <= 15")
    print(f"{str(nama)} memiliki banyak question: {len(q_sixteen_to_twenty)} dengan panjang 16 <= x <= 20")
    print(f"{str(nama)} memiliki banyak question: {len(q_over_twenty)} dengan panjang > 20")
    print("=" * 100)
    print(f"{str(nama)} memiliki banyak question: {len(a_zero)} dengan panjang 0")
    print(f"{str(nama)} memiliki banyak question: {len(a_one_to_five)} dengan panjang 1 <= x <= 5")
    print(f"{str(nama)} memiliki banyak question: {len(a_six_to_ten)} dengan panjang 6 <= x <= 10")
    print(f"{str(nama)} memiliki banyak question: {len(a_eleven_to_fifteen)} dengan panjang 11 <= x <= 15")
    print(f"{str(nama)} memiliki banyak question: {len(a_sixteen_to_twenty)} dengan panjang 16 <= x <= 20")
    print(f"{str(nama)} memiliki banyak question: {len(a_over_twenty)} dengan panjang > 20")

In [22]:
statistics_length(data=squad_id_train_df, nama="SQUAD-ID Train")
statistics_length(data=squad_id_validation_df, nama="SQUAD-ID Validation")
statistics_length(data=squad_id_all_df, nama="SQUAD-ID All")
print()
statistics_length(data=data_idk_train_df, nama="IDK-MRC Train")
statistics_length(data=data_idk_validation_df, nama="IDK-MRC Validation")
statistics_length(data=data_idk_all_df, nama="IDK-MRC All")
print()
statistics_length(data=data_tydiqa_train_df, nama="TYDIQA Train")
statistics_length(data=data_tydiqa_validation_df, nama="TYDIQA Validation")
statistics_length(data=data_tydiqa_all_df, nama="TYDIQA All")

SQUAD-ID Train memiliki banyak context: 68521 dengan panjang < 100
SQUAD-ID Train memiliki banyak context: 61798 dengan panjang 101 <= x <= 150
SQUAD-ID Train memiliki banyak context: 0 dengan panjang 151 <= x <= 200
SQUAD-ID Train memiliki banyak context: 0 dengan panjang 201 <= x <= 250
SQUAD-ID Train memiliki banyak context: 0 dengan panjang 251 <= x <= 300
SQUAD-ID Train memiliki banyak context: 0 dengan panjang > 300
SQUAD-ID Train memiliki banyak question: 17332 dengan panjang 1 <= x <= 5
SQUAD-ID Train memiliki banyak question: 112987 dengan panjang 6 <= x <= 10
SQUAD-ID Train memiliki banyak question: 0 dengan panjang 11 <= x <= 15
SQUAD-ID Train memiliki banyak question: 0 dengan panjang 16 <= x <= 20
SQUAD-ID Train memiliki banyak question: 0 dengan panjang > 20
SQUAD-ID Train memiliki banyak question: 9 dengan panjang 0
SQUAD-ID Train memiliki banyak question: 115151 dengan panjang 1 <= x <= 5
SQUAD-ID Train memiliki banyak question: 15159 dengan panjang 6 <= x <= 10
SQUAD-I

In [23]:
def question_type(data, nama):
    num_apa = 0
    num_dimana = 0
    num_kapan = 0
    num_siapa = 0
    num_bagaimana = 0
    num_kenapa = 0
    num_berapa = 0
    num_others = 0
    
    for i in data['question']:
        current_question = i.split()
        
        if 'Apa' in current_question: num_apa += 1
        elif 'Apakah' in current_question:num_apa += 1
        elif 'apa' in current_question:num_apa += 1
        elif 'apakah' in current_question:num_apa += 1
        
        elif 'Dimana' in current_question: num_dimana += 1
        elif 'dimana' in current_question: num_dimana += 1
        elif 'mana' in current_question: num_dimana += 1
        
        elif 'Kapan' in current_question: num_kapan += 1
        elif 'kapan' in current_question: num_kapan += 1
        
        elif 'Siapa' in current_question: num_siapa += 1
        elif 'siapa' in current_question: num_siapa += 1
        
        elif 'Bagaimana' in current_question: num_bagaimana += 1
        elif 'bagaimana' in current_question: num_bagaimana += 1

        elif 'Mengapa' in current_question: num_kenapa += 1
        elif 'Kenapa' in current_question: num_kenapa += 1
        elif 'mengapa' in current_question: num_kenapa += 1
        elif 'kenapa' in current_question: num_kenapa += 1
        
        elif 'Berapa' in current_question: num_berapa += 1
        elif 'Berapakah' in current_question: num_berapa += 1
        elif 'berapa' in current_question: num_berapa += 1
        elif 'berapakah' in current_question: num_berapa += 1
        
        else: num_others += 1
        
    denominator = len(data['question'])
    num_apa_percentage = num_apa / denominator
    num_dimana_percentage = num_dimana / denominator
    num_kapan_percentage = num_kapan / denominator
    num_siapa_percentage = num_siapa / denominator
    num_kenapa_percentage = num_kenapa / denominator
    num_bagaimana_percentage = num_bagaimana / denominator
    num_berapa_percentage = num_berapa / denominator
    num_others_percentage = num_others / denominator
    
    print(f"{str(nama)} memiliki banyak pertanyaan APA sekitar: {num_apa}")
    print(f"{str(nama)} memiliki banyak pertanyaan DIMANA sekitar: {num_dimana}")
    print(f"{str(nama)} memiliki banyak pertanyaan KAPAN sekitar: {num_kapan}")
    print(f"{str(nama)} memiliki banyak pertanyaan SIAPA sekitar: {num_siapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan KENAPA sekitar: {num_kenapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan BAGAIMANA sekitar: {num_bagaimana}")
    print(f"{str(nama)} memiliki banyak pertanyaan BERAPA sekitar: {num_berapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan LAINNYA sekitar: {num_others}")
    print()
    
    #assert num_apa_percentage+num_dimana_percentage+num_kapan_percentage+num_siapa_percentage+num_kenapa_percentage+num_bagaimana_percentage+num_berapa_percentage+num_others_percentage == 1.0
        
    print(f"{str(nama)} memiliki banyak pertanyaan APA sekitar: {num_apa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan DIMANA sekitar: {num_dimana_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan KAPAN sekitar: {num_kapan_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan SIAPA sekitar: {num_siapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan KENAPA sekitar: {num_kenapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan BAGAIMANA sekitar: {num_bagaimana_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan BERAPA sekitar: {num_berapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan LAINNYA sekitar: {num_others_percentage * 100} %")

In [24]:
question_type(data=squad_id_train_df, nama="SQUAD-ID Train")
question_type(data=squad_id_validation_df, nama="SQUAD-ID Validation")
question_type(data=squad_id_all_df, nama="SQUAD-ID All")
print()
question_type(data=data_idk_train_df, nama="IDK-MRC Train")
question_type(data=data_idk_validation_df, nama="IDK-MRC Validation")
question_type(data=data_idk_all_df, nama="IDK-MRC All")
print()
question_type(data=data_tydiqa_train_df, nama="TYDIQA Train")
question_type(data=data_tydiqa_validation_df, nama="TYDIQA Validation")
question_type(data=data_tydiqa_all_df, nama="TYDIQA All")

SQUAD-ID Train memiliki banyak pertanyaan APA sekitar: 62856
SQUAD-ID Train memiliki banyak pertanyaan DIMANA sekitar: 11812
SQUAD-ID Train memiliki banyak pertanyaan KAPAN sekitar: 7696
SQUAD-ID Train memiliki banyak pertanyaan SIAPA sekitar: 13340
SQUAD-ID Train memiliki banyak pertanyaan KENAPA sekitar: 1884
SQUAD-ID Train memiliki banyak pertanyaan BAGAIMANA sekitar: 2972
SQUAD-ID Train memiliki banyak pertanyaan BERAPA sekitar: 18438
SQUAD-ID Train memiliki banyak pertanyaan LAINNYA sekitar: 11321

SQUAD-ID Train memiliki banyak pertanyaan APA sekitar: 48.23241430643267 %
SQUAD-ID Train memiliki banyak pertanyaan DIMANA sekitar: 9.063912399573354 %
SQUAD-ID Train memiliki banyak pertanyaan KAPAN sekitar: 5.905508789969229 %
SQUAD-ID Train memiliki banyak pertanyaan SIAPA sekitar: 10.236419862030862 %
SQUAD-ID Train memiliki banyak pertanyaan KENAPA sekitar: 1.4456832848625296 %
SQUAD-ID Train memiliki banyak pertanyaan BAGAIMANA sekitar: 2.280557708392483 %
SQUAD-ID Train memiliki